# Funciones hash

### Método de división

$$ h(x) = |x| \text{mod} M $$

Es muy recomendable utilizar un número primo para $M$.

In [150]:
def division(x=0, M=97):
    return abs(x) % M

print(division(30501))
print(division(30502))
print(division(-30501))
for i in range(10):
    print(i, '=>', division(i))

43
44
43
0 => 0
1 => 1
2 => 2
3 => 3
4 => 4
5 => 5
6 => 6
7 => 7
8 => 8
9 => 9


### Método de la mitad del cuadrado

$$ h(x) = \frac{W}{M}x^2 \text{mod} W $$

Con $W$ igual al tamaño de la palabra de la máquina, por ejemplo, $W=2^{32}$.

In [151]:
def mitad_cuadrado(x=0, k=10, w=32):
    mask = (1 << k) - 1
    return ((x**2) >> (w-k)) & mask

print(mitad_cuadrado(30501))
print(mitad_cuadrado(30502))
print(mitad_cuadrado(-30501))
for i in range(10):
    print(i, '=>', mitad_cuadrado(i))

221
221
221
0 => 0
1 => 0
2 => 0
3 => 0
4 => 0
5 => 0
6 => 0
7 => 0
8 => 0
9 => 0


### Método de Fibonacci

$$ F_{n+2} = F_{n+1} + F_n \qquad \Rightarrow \qquad F_{n+2} - F_{n+1} - F_{n} = 0$$

$$ \qquad \qquad \qquad \quad \Rightarrow \qquad \phi^2-\phi-1=0$$

$$ \qquad \qquad \qquad \Rightarrow \quad \phi_{1,2} = \frac{1 \pm 5}{2}$$

El $n$-ésimo número de Fibanacci es de la forma:

$$ f_n = \frac{1}{\sqrt{5}}(\phi^n_1 - \phi^n_2)$$

In [152]:
from math import sqrt
((1+sqrt(5))/2)**2

2.618033988749895

In [153]:
(1-sqrt(5))/2

-0.6180339887498949

In [154]:
def fibonacci(x=0, k=10, w=32, a=2654435769):
    mask = (1 << k) - 1
    return ((x * a) >> (w-k)) & mask

print(fibonacci(30501))
print(fibonacci(30502))
print(fibonacci(-30501))
for i in range(10):
    print(i, '=>', fibonacci(i))

670
279
353
0 => 0
1 => 632
2 => 241
3 => 874
4 => 483
5 => 92
6 => 725
7 => 334
8 => 966
9 => 575


# Manipulando datos faltantes

In [155]:
import pandas as pd
from io import StringIO

In [156]:
datos = \
"""A,B,C,D
1.0,2.1,3.7,
5.9,6.2,,8.6
9.3,0.4,1.8"""
df = pd.read_csv(StringIO(datos))
df

,A,B,C,D
0,1.0,2.1,3.7,NaN
1,5.9,6.2,NaN,8.6
2,9.3,0.4,1.8,NaN


In [157]:
df.isnull().sum()

A    0
B    0
C    1
D    2
dtype: int64

### Eliminando valores faltantes

Una de las maneras más simples de manipular los valores faltantes es eliminar características (columnas, features) o muestras (filas) que contengan valores faltantes.

In [158]:
df.dropna(axis=1) # Por columna

,A,B
0,1.0,2.1
1,5.9,6.2
2,9.3,0.4


In [159]:
df.dropna(axis=0) # Por fila

,A,B,C,D


In [160]:
df.dropna(how='all') # Elimina las muestras en las que todas sus columnas son NaN

,A,B,C,D
0,1.0,2.1,3.7,NaN
1,5.9,6.2,NaN,8.6
2,9.3,0.4,1.8,NaN


In [161]:
df.dropna(thresh=4) # Elimina las muestras que tengan menos de 4 valores no NaN

,A,B,C,D


In [162]:
df.dropna(subset=['C']) # Elimina las muestras que tengan NaN en la columna C

,A,B,C,D
0,1.0,2.1,3.7,NaN
2,9.3,0.4,1.8,NaN


In [163]:
df.dropna(subset=['A','C'])

,A,B,C,D
0,1.0,2.1,3.7,NaN
2,9.3,0.4,1.8,NaN


### Sustitución de valores faltantes

En ocasiones no es recomendable eliminar características o muestras completas debido a que se podrían perder datos valiosos en el proceso; en este caso se pueden utilizar técnicas de interpolación para estimar los valores faltantes a partir de otras muestras en nuestro conjunto de datos. Una usada comúnmente es la media por características (columnas).

Sustituye valores faltantes con la media

In [164]:
import numpy as np

In [165]:
datos_n = df.copy() # No afectar al df original
for col in df.columns.values :
    falta = np.sum(df[col].isnull())
    if falta:
        print ('Asignando {} valores en columna : {} '.format(falta,col))
        mean = df[col].mean()
datos_n[col] = df[col].fillna(mean)

Asignando 1 valores en columna : C 
Asignando 2 valores en columna : D 


Se puede utilizar la clase *SimpleImputer* para sustituir datos faltantes:

In [166]:
from sklearn.impute import SimpleImputer

In [167]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(df.values)
datos_n = imp.transform(df.values)
datos_n

array([[1.  , 2.1 , 3.7 , 8.6 ],
       [5.9 , 6.2 , 2.75, 8.6 ],
       [9.3 , 0.4 , 1.8 , 8.6 ]])

Otras opciones para el parámetro *strategy* son *median* y *most_frequent*, este último coloca el
valor más repetido en lugar de los faltantes y es muy útil cuando se tienen datos categóricos:

In [168]:
df = pd.DataFrame([['a', np.nan],
                  ['b', 'y'],
                  ['c', 'x'],
                  ['a', 'y'],
                  [np.nan, 'z']], dtype='category')

imp = SimpleImputer(strategy='most_frequent')
imp.fit_transform(df)

array([['a', 'y'],
       ['b', 'y'],
       ['c', 'x'],
       ['a', 'y'],
       ['a', 'z']], dtype=object)